# Lightning Channel via Batch Transaction

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup LND GRPC

In [ ]:
import sys

In [ ]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [ ]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [ ]:
from lnd_rpc import LndRpc, lnmsg, walletmsg

In [ ]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [ ]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [ ]:
ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey

In [ ]:
bob.lnd.GetInfo(lnmsg.GetInfoRequest())

## Mine coins

In [ ]:
import os
import bitcoin
from rpc import Proxy, Config

In [ ]:
bitcoin.SelectParams('regtest')

In [ ]:
rpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [ ]:
rpc.getblockcount()

In [ ]:
aliRpc.wallet.ListUnspent(walletmsg.ListUnspentRequest())

In [ ]:
aliRpc.lnd.GetInfo(lnmsg.GetInfoRequest())